## Graph Database Q&A Chatbot

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url= NEO4J_URI, username= NEO4J_USERNAME, password= NEO4J_PASSWORD)
graph

In [4]:
# Dataset Movie Review


movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row
    
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [5]:
movies_query

"\nLOAD CSV WITH HEADERS FROM \n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row\n    \nMERGE (m:Movie {id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n"

In [6]:
graph.query(movies_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [8]:
# LLM

from langchain_groq import ChatGroq

llm = ChatGroq(model= 'llama3-8b-8192', api_key= GROQ_API_KEY)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x15f5214e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x15f5221a0>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [9]:
from langchain.chains.graph_qa.cypher import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm= llm, 
    graph= graph,
    verbose= True
)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x106f06440>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x15f5214e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x15f5221a0>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))), qa_chain=LLMChai

In [11]:
response = chain.invoke({"query":"Who was the director of the movie 'Jumanji'"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE m.title = 'Jumanji' RETURN p.name
Full Context:
[{'p.name': 'Joe Johnston'}]

> Finished chain.


{'query': "Who was the director of the movie 'Jumanji'",
 'result': "Joe Johnston was the director of the movie 'Jumanji'."}

In [13]:
response = chain.invoke({"query": "Who played in GoldenEye?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = "GoldenEye" RETURN p.name;
Full Context:
[{'p.name': 'Pierce Brosnan'}, {'p.name': 'Famke Janssen'}, {'p.name': 'Sean Bean'}, {'p.name': 'Izabella Scorupco'}]

> Finished chain.


{'query': 'Who played in GoldenEye?',
 'result': 'Pierce Brosnan, Famke Janssen, Sean Bean, Izabella Scorupco played in GoldenEye.'}

In [15]:
# Return intermediate results

chain = GraphCypherQAChain.from_llm(
    llm= llm, graph=graph, verbose=True, return_intermediate_steps=True
)

In [16]:
result = chain.invoke({"query": "Who played in the movie 'GoldenEye'?"})
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:"GoldenEye"}) RETURN p.name;
Full Context:
[{'p.name': 'Pierce Brosnan'}, {'p.name': 'Famke Janssen'}, {'p.name': 'Sean Bean'}, {'p.name': 'Izabella Scorupco'}]

> Finished chain.
Intermediate steps: [{'query': 'MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:"GoldenEye"}) RETURN p.name;'}, {'context': [{'p.name': 'Pierce Brosnan'}, {'p.name': 'Famke Janssen'}, {'p.name': 'Sean Bean'}, {'p.name': 'Izabella Scorupco'}]}]
Final answer: Pierce Brosnan, Famke Janssen, Sean Bean, Izabella Scorupco played in the movie 'GoldenEye'.
